<a href="https://colab.research.google.com/github/nikolmash/autobreya/blob/main/hw2/hw2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Извлечение коллокаций + NER

В качестве корпуса я использовала отзывы с Amazon на одежду и аксессуары 

In [1]:
!wget https://www.dropbox.com/s/awgoua1rp1i871d/Clothing_%26_Accessories.txt

--2020-12-21 12:37:28--  https://www.dropbox.com/s/awgoua1rp1i871d/Clothing_%26_Accessories.txt
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/awgoua1rp1i871d/Clothing_%26_Accessories.txt [following]
--2020-12-21 12:37:28--  https://www.dropbox.com/s/raw/awgoua1rp1i871d/Clothing_%26_Accessories.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc25aca8517d2b5212aa15cd4655.dl.dropboxusercontent.com/cd/0/inline/BFfIPwhrN48mjkrD0uxWln61yR88iaNsS9P_BbJKz6hdqGOuCHUmgR6_oyOei4O6F0EssyFsIhtj_UT5MYucQkeMXJwtHv6qMQ6nBmE8NJ28-SX6HmKI43SlNJxreOnMxXs/file# [following]
--2020-12-21 12:37:28--  https://uc25aca8517d2b5212aa15cd4655.dl.dropboxusercontent.com/cd/0/inline/BFfIPwhrN48mjkrD0uxWln61yR88iaNsS9P_BbJKz6hdqGOuCHUmgR6_oyOei4O

In [2]:
import re

filename = "Clothing_&_Accessories.txt"

reviews = []
with open(filename) as f:
    f = f.readlines()
for l in f:
    l = l.strip()
    if re.match('review/text:', l):
        start_ind = len('review/text: ')
        reviews.append(l[start_ind:])
reviews = list(set(reviews))

## 1.

**Первый способ:**

Вычитав где-то 100 отзывов, я заметила, что в текстах часто повторяются одни и те же фразы/выражения, из которых достаточно легко с помощью правил вытащить сущность товара. Это либо оценочные суждения, либо констатация факта (я заказал, купил):
- *I bought/ purchaced/ received this (dress)*
- *Love/enjoy this (shirt)*
- различные словоформы лексемы *wear (I wear this jeans every day/I have worn this ring for a year)*
- *what a (beautiful dress)*
- *one of the best (pair of pants) I have had*
- this is (comfortable jeans)

В силу того, что в такой категории текстов используется упрощенная лексика, отсутствие каких-либо художественных средств, действительно, такие предложения могут встречаться почти в каждом отзыве. Поэтому будем использовать их как шаблоны bootstrapping и будем находить названия товары с помощью задания правил вручную.

Минус такого подхода в том, что, естесственно, такие шаблонные фразы есть далеко не во всех отзывах, и какую-то часть информации мы упустим. Очень часто также в отзывах эксплицитно не называется товар, а заменяется местоимением it, но при этом такая сущность все равно будет подходить под шаблон, и нам надо будет все подчищать, в зависимости от части речи. И еще есть проблема с размытыми границами. Словит ли шаблон употребление (blue jeans вместо beatiful blue jeans)? А если нет, являются ли два этих упоминания товаров различными для нашего отчета? Это проблема в принципе актуальна для всех методов, и такие вещи нужно заранее обговаривать с заказчиком, если речь идет о реальной задаче


**Второй способ:**

Помимо того, что у нас есть текст самого отзыва, также у каждого товара есть заголовок. Конкретно для корпуса с отзывами на одежду и аксессуары, скорее всего в названии будет не сам item одежды, но и цвет, стиль, бренд, узор. Поэтому просто проверять вхождение заголовка в отзыв - плохая идея. Здесь можно использовать векторные представления, например, word2vec. Чтобы модель сконцентрировалась именно на нашей задаче, можно с помощью библиотеки gensim обучить свой word2vec на всех отзывах и на всех названиях товаров. И затем для каждого названия (желательно убрать оттуда с помощью морфологического анализа все, что не существительное, еще и неодушевленное) искать наиболее близкие слова в отзывах. 

Если не использовать заголовки (это возможно только усложнит задачу, потому что некоторые наименования товаров очень длинные, содержат много ненужной информации), то можно извлечь с сайта любого интернет-магазина категории товаров (одежды и аксессуары), и искать похожие слова на них. Но тогда мы не будем знать какой отзыв соответствует какой категории. Будет больше вычислений.

Минусы: word2vec может не сделать чуда, и не находить что-то внятно похожее на название/категорию товаров. Остается проблема с отзывами, где нет прямого обозначения товара, есть только it. Сложно с точки зрения вычислимости.

**Третий способ:**

Использовать BERT для задачи NER или просто нейросеть (biLSTM-CRF), которая будет предсказывать NE в тексте отзыва. Самая большая проблема в том, что нужен какой-то датасет для обучения. В принципе можно использовать простые методы, как выше, чтобы нагенерировать примеры, подчистить, и подать на вход модели, которая сделает все за нас. Можно также добавить какие-то признаки, например, pos-разметку (сущность скорее всего - существительное),  синтаксическую информацию (из шаблонов выше видно, что сущность чаще всего либо подлежащее либо прямое дополнение). 

Минусы: как я уже написала, нужны размеченные данные. Обучать что-либо достаточно долго, и надо пробовать

**Четвертый способ**: 

Довольно примитивный, но мне внезапно пришла в голову эта идея и я решила добавить. Для каждого отзыва указан id продукта, для которого он написан. А что, если сгруппировать отзывы по каждому продукту (а лучше вообще для каждой категории товаров, если это как-то закодировано в id). Получится, что для каждого товара у нас имеется один большой отзыв, и внутри него могут быть совершенно разные, написанные разными людьми упоминания одного и того же товара. Среди них можно посмотреть на биграммы с высокой совместной встречаемостью, или для начала просто на самые частотные существительные. Я думаю, что какой-либо предмет одежду/аксессуаров имеет достаточно ограниченное количество слов, которым их можно назвать. А уже от полученных существительных, или биграмм, можно искать в помощью синтаксической разметки зависящие от них единицы (какую-то целую составляющую в дереве. Например, если мы нашли с помощью частых биграмм christmas socks, то обязательно найдем nice christmas socks).

Минусы: нужно все просматривать глазами, все чистить от лишнего. Использование нетривиальной синтаксической разметки, тем более предложения могут быть грамматически неполными, если это какой-то короткий отзыв, и парсер на них сломается. Уникальные упоминания товаров (а вдруг такие будут) не будут являться частотными, а значит, не попадут в наше рассмотрение. С другой стороны, топ самых редких слов (особенно только существительных) просмотреть - не очень большая проблема.

## 2. 

Я буду реализовывать первый метод с помощью вручную заданных правил/шаблонов

В задании указан совет делать это через natasha/yargy, что очень странно, поскольку он написан для русского языка (внутри него лежит pymorphy, можно, наверное писать простые правила для английского и без функционала морфологии, но кажется, это не очень рационально). Поэтому я использую похожий [инструмент](https://spacy.io/usage/rule-based-matching#matcher) из spacy

In [3]:
import spacy
from spacy.matcher import Matcher

Шаблоны пишутся с помощью ограничений на токены (вернее, на то, как их разбирает spacy): на их леммы, морфологические теги и т.д. 

Исходя из тех частотных фраз, что я выделила ранее, создадим три шаблона:
1. название продукта является прямым объектом глаголов, которые выражают оценочное суждение (*love, enjoy, like*) или глаголов, доказывающих факт покупки товара (*buy, purchase, get*). У названия продукта также может быть артикль и различные модификаторы в виде прилагательных и существительных
2. чисто оценочные фразы типа *this is a nice shirt* или *it is the best socks*. Здесь название продукта входит в составное сказуемое, состоящее также из вспомогательного глагола *be*. Также учитывается неограниченное кол-во модификаторов
3. Фразы *what a beautiful dress!*. Здесь название продукта является непосредственно корнем дерева, а *what* - его очередным детерминантом.

Понятно, что этими тремя шаблонами, конечно, задача не ограничивается. Но эти мне показались самыми частотными и легко выделяемыми.

In [4]:
nlp = spacy.load("en_core_web_sm")
matcher = Matcher(nlp.vocab)
pattern1 = [{'LEMMA': {"IN": ['buy','receive','purchase','order','love','enjoy','like','wear' ]}},
           {'POS': 'DET', 'OP': '?'},
           {'DEP': 'amod', 'OP': '*'},
           {'POS': 'NOUN', 'OP': '*'},
           {'DEP': 'dobj', 'POS': 'NOUN'}]
matcher.add('verb+dobj', None, pattern1)

In [5]:
pattern2 = [{'LEMMA': {'IN': ['this', '-PRON-']}, 'DEP': 'nsubj'},
            {'LEMMA': 'be', 'POS': 'AUX', 'TAG': 'VBZ'},
            {'POS': 'DET', 'OP': '?'},
            {'DEP': 'amod', 'OP': '*'},
            {'POS': 'NOUN', 'OP': '*'},
            {'DEP': 'attr', 'POS': 'NOUN'}
            ]
matcher.add('this+attr', None, pattern2)

In [6]:
pattern3 = [{'LEMMA': 'what', 'DEP': 'det'},
            {'POS': 'DET', 'OP': '?'},
            {'DEP': 'amod', 'OP': '*'},
            {'POS': 'NOUN', 'OP': '*'},
            {'DEP': 'ROOT', 'POS': 'NOUN'}
            ]
matcher.add('what+root', None, pattern3)

Посмотрим, что за части отзывов выделяются по этим шаблонам:

In [7]:
for rev in reviews[:200]:
  doc = nlp(rev)
  matches = matcher(doc)
  for match_id, start, end in matches:
    span = doc[start:end]
    print(span.text)

ordered this bag
like the beach
It is prada
it is good quality
ordered a 4X
This is a great bra
purchase this bag
enjoy this bag
ordered my bra
love this bra
buying this bra
This is a neccessary article
wear pants
loves his red chair
bought a pair
wear this shoe
love the size
worn these things
ordered this pair
wear this size
purchased a bunch
wear Black caps
loves this costume
this is a minimizer
It's my favorite style
love this wallet
It is the perfect size
it is a great sports bra
this is a great cause
received this bag
love the square zipper
This is a great costume
received the box
receive the item
receive a credit
love the waterproof part
purchased a bra
This is a wonderful item
purchase these shirts
bought additional pair
buy Levis
bought a few pairs
liked the way
wearing my ID
bought this wallet
bought this one
loves this belt
received item
Received many many compliments
order the size
wear 12"-shaft western boots
This is casual wear
This is the softest robe
This is a great bath

Выделяется примерно то, что и планировалось. И даже если у названия продукта есть какие-то модификации в виде прилагательных или других существительных типа *leather*, они тоже были выделены. Естественно, не все примеры идеальны - например, шаблон цепляет фразы про размер продукта, про его форму/качество, поскольку все эти характеристики также представлены в виде существительных (*size, style, length, quality, cotton*). Также стоит сделать скидку на то, что не все разборы spacy идеальны, а в шаблонах есть достаточно сильная привязка к морфосинтаксическим тегам.

По сути, из большого отзыва мы научились вытаскивать микро-предложение, поскольку все, что подходит под шаблон, грамматически им является. Соответственно, из этих вычлененных частей будет легко достать само название продукта, опять же с помощью синтаксических тегов spacy, которые будут корректными в силу полноты предложения.

Было бы удобно прописывать в правилах, как в yargy, какое из правил относится к какой сущности, и сразу их маркировать, тогда можно было бы отдельно доставать глагол, отдельно модификации, отдельно - сам продукт. Но, к сожалению, такой функционал не предусмотрен.



Будем работать с каждым шаблоном по отдельности, потому что в каждом шаблоне само название продуктов имеет разную синтаксическую роль.

В первом шаблоне продуктом является прямой объект. У него также может быть важный модификатор, без которого единственное существительное не является названием продуктом. Такие модификаторы spacy помечает как compound (см.ниже в дереве *phone cover* - полное название продукта). При наличии, его тоже будем извлекать. Модификации в виде прилагательных не будем извлекать, потому что они часто оценочные. Их проанализируем ниже, когда будем смотреть на соседей

In [8]:
from spacy import displacy
from IPython.core.display import display, HTML

doc1 = nlp('bought this phone cover')
html = displacy.render([doc1], style="dep")
display(HTML(html))

В рамках задания я возьму не полный датасет из почти 600 000 отзывов, а только 100 000. Matcher из spacy работает очень долго, а этого объема текстов должно хватить для ранжирования

In [9]:
matcher1 = Matcher(nlp.vocab)
matcher1.add('verb+dobj', None, pattern1)

matched_to_pattern1 = []

for rev in reviews[:100000]:
  doc = nlp(rev)
  matches = matcher1(doc)
  for match_id, start, end in matches:
    span = doc[start:end]
    matched_to_pattern1.append(span.text)

In [10]:
print(len(matched_to_pattern1))

40670


В список products будем записывать вытянутые упоминания товаров

In [11]:
matcher1_to_products = Matcher(nlp.vocab)
pattern1_products = [{'DEP': 'compound', 'POS': 'NOUN', 'OP': '*'},
                     {'DEP': 'dobj', 'POS': 'NOUN'}]
matcher1_to_products.add('dobj', None, pattern1_products)

products = []
for micro_sent in matched_to_pattern1:
  doc = nlp(micro_sent)
  matches = matcher1_to_products(doc)
  for match_id, start, end in matches:
    span = doc[start:end]
    if span.text not in products:
      products.append(span.text)

Проделаем то же самое со вторым шаблоном, где искомое название товара является частью составного сказуемого (тег attr)

In [12]:
matcher2 = Matcher(nlp.vocab)
matcher2.add('this+attr', None, pattern2)

matched_to_pattern2 = []

for rev in reviews[:100000]:
  doc = nlp(rev)
  matches = matcher2(doc)
  for match_id, start, end in matches:
    span = doc[start:end]
    matched_to_pattern2.append(span.text)

In [13]:
print(len(matched_to_pattern2))

10947


In [14]:
matcher2_to_products = Matcher(nlp.vocab)
pattern2_products = [{'DEP': 'compound', 'POS': 'NOUN', 'OP': '*'},
                     {'DEP': 'attr', 'POS': 'NOUN'}]
matcher2_to_products.add('attr', None, pattern2_products)

for micro_sent in matched_to_pattern2:
  doc = nlp(micro_sent)
  matches = matcher2_to_products(doc)
  for match_id, start, end in matches:
    span = doc[start:end]
    if span.text not in products:
      products.append(span.text)

И те же самые действия с третьим шаблоном, где наименование товара является корнем дерева зависимостей

In [15]:
matcher3 = Matcher(nlp.vocab)
matcher3.add('what+root', None, pattern3)

matched_to_pattern3 = []

for rev in reviews[:100000]:
  doc = nlp(rev)
  matches = matcher3(doc)
  for match_id, start, end in matches:
    span = doc[start:end]
    matched_to_pattern3.append(span.text)

In [16]:
print(len(matched_to_pattern3))

367


Видно, что этот шаблон не самый удачный, но тем не менее и оттуда вычленим упоминания:

In [17]:
matcher3_to_products = Matcher(nlp.vocab)
pattern3_products = [{'DEP': 'compound', 'POS': 'NOUN', 'OP': '*'},
                     {'DEP': 'ROOT', 'POS': 'NOUN'}]
matcher3_to_products.add('root', None, pattern3_products)

for micro_sent in matched_to_pattern3:
  doc = nlp(micro_sent)
  matches = matcher3_to_products(doc)
  for match_id, start, end in matches:
    span = doc[start:end]
    if span.text not in products:
      products.append(span.text)

Упоминания товаров из отзывов извлечены. Посмотрим на случайные элементы итогового списка products: 

In [18]:
print('Количество упоминаний: ', len(products))

Количество упоминаний:  5631


In [19]:
import numpy as np

for rand_ind in np.random.randint(len(products), size=30):
  print(products[rand_ind])

sleeveless versions
layer socks
bearcat
bootie
workmanship
quality mask
heater
glamorise
brain bucket
bandana-
compression bra
series
oil
barrier
lines
firefighter
washcloths
love
skullcap
duplicate
instructions
space captain!Great item
pillows
boxer shorts
beatles
cycling shorts
hair fork
childs
indication
gauche


##3. 

Для коллокаций будем смотреть на левого соседа. Это более естесственно для английского языка (в именной группе сначала идут модификаторы) и наши шаблоны были составлены именно так.

Для этого мы пройдемся по тем частям отзывов, что подошли под наши шаблоны

In [20]:
matched_to_patterns = matched_to_pattern1
matched_to_patterns.extend(matched_to_pattern2)
matched_to_patterns.extend(matched_to_pattern3)

In [21]:
ngrams = []

for match in matched_to_patterns:
  for ent in products:
    if ent in match:
      match_words = match.split()
      ent_words = ent.split()
      if ent_words[0] in match_words:
        ent_ind = match_words.index(ent_words[0])
        if ent_ind>0:
          ngrams.append(' '.join(match_words[ent_ind-1:ent_ind+len(ent)]))

Посмотрим на случайные 30 n-грамм:

In [22]:
for rand_ind in np.random.randint(len(ngrams), size=30):
  print(ngrams[rand_ind])

the support
these shorts
regular glasses
shoe compartment
this style
a diesal mechanic
this hat
a pair
these shirts
these socks
cell phone case
this product
my 3rd shirt
wear boxers
this costume
a second pair
hot chillys
a couple
the dresses
the ability
fine shirt
this bag
this bag
ordered extras
receive compliments
poor design
different kinds
decent flannel
the zippers
another pair


Очень часто левым соседом наименования сущности является артикль, что скорее всего говорит об отсутствии какого-то модификатора, поэтому никакой значимой информации мы не теряем. Есть достаточно много интересных коллокаций, перейдем к их анализу

##4.

Выделенные n-граммы различаются по длине, поэтому чтобы применять коллокационные метрики, надо их разделить. Посмотрим сначала, какое количество биграмм, триграмм и т.д. имеется:

In [23]:
lengthes = {2: 0, 3: 0, 4: 0, 'more': 0}
for x in ngrams:
  if len(x.split())>4:
    lengthes['more'] += 1
  else:
    lengthes[len(x.split())] += 1

lengthes

{2: 51424, 3: 12002, 4: 1086, 'more': 97}

В основном биграммы и триграммы, а а также немного квадграмм, которые не будем брать во внимание. Разделим би- и триграммы на отдельные списки

In [24]:
bigrams = []
trigrams = []

for x in ngrams:
  if len(x.split()) == 2:
    bigrams.append(x.split())
  elif len(x.split()) == 3:
    trigrams.append(x.split())


In [25]:
import nltk 
from nltk.collocations import *

bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()

finder2 = BigramCollocationFinder.from_documents(bigrams)
finder3 = TrigramCollocationFinder.from_documents(trigrams)

Применим частотные фильтры: для биграм как минимум три упоминания (так как их больше), для триграм - два 

In [26]:
finder2.apply_freq_filter(3)
finder3.apply_freq_filter(2)

Получим топ-20 ранжированных коллокаций различными способами:

In [27]:
finder2.nbest(bigram_measures.student_t, 20)

[('a', 'size'),
 ('this', 'shirt'),
 ('this', 'bra'),
 ('this', 'item'),
 ('these', 'socks'),
 ('this', 'bag'),
 ('this', 'product'),
 ('a', 'pair'),
 ('this', 'costume'),
 ('another', 'one'),
 ('this', 'hat'),
 ('another', 'pair'),
 ('the', 'way'),
 ('these', 'pants'),
 ('these', 'jeans'),
 ('a', 'medium'),
 ('the', 'fact'),
 ('this', 'dress'),
 ('the', 'color'),
 ('sports', 'bra')]

Нет сомнений, что Критерий Стьюдента ранжирует высокими действительно часто встречаемые коллокации, потому что существительные сопровождаются артиклем. Также можно заметить, что среди топа немного "мусора" - только size, way, medium, fact и color. Но эти коллокации не являются информативными для отчета, поэтому эта метрика не подходит для конкретной задачи

In [28]:
finder2.nbest(bigram_measures.pmi, 20)

[('arch', 'supports'),
 ('under', 'armour'),
 ('ear', 'muffs'),
 ('false', 'advertising'),
 ('conversation', 'starter'),
 ('eel', 'skin'),
 ('lottery', 'ticket'),
 ('star', 'wars'),
 ('olive', 'drab'),
 ('rash', 'guard'),
 ('eye', 'catcher'),
 ('rave', 'reviews'),
 ('THESE', 'GLASSES'),
 ('THESE', 'SOCKS'),
 ('burp', 'cloths'),
 ('super', 'heros'),
 ('thigh', 'highs'),
 ('bouncy', 'seat'),
 ('average', 'height'),
 ('synthetic', 'fibers')]

У метрики PMI есть явная склонность к высокому ранжированию коллокаций, состоящих из двух существительных (в рамках этого датасета и моего метода). Причем среди топ-20 очень мало того, что относится к названиям продуктов. По сравнению с предыдущей метрикой, где лишнее было достаточно просто почистить руками, здесь высоко ранжируется много всего странного. А явные названия товаров высоко ранжированы только с детерминантом this. Эта метрика точно не подойдет для отчета

In [29]:
finder2.nbest(bigram_measures.chi_sq, 20)

[('rash', 'guard'),
 ('arch', 'supports'),
 ('ear', 'muffs'),
 ('conversation', 'starter'),
 ('star', 'wars'),
 ('false', 'advertising'),
 ('receiving', 'blankets'),
 ('leg', 'warmers'),
 ('olive', 'drab'),
 ('under', 'armour'),
 ('thigh', 'highs'),
 ('eye', 'catcher'),
 ('rave', 'reviews'),
 ('eel', 'skin'),
 ('lottery', 'ticket'),
 ('many', 'compliments'),
 ('super', 'heros'),
 ('THIS', 'PRODUCT'),
 ('card', 'holder'),
 ('adjustable', 'straps')]

Хи-квадрат выдает результат, похожий на выдачу ранжирования с метрикой PMI. По крайней мере набор биграмм такой же, немного отличается порядок. Опять же высоко ранжированы какие-то составные названия (rash guard, например, это наименование спортивной кофты). Но нет никаких коллокаций с оценочными прилагательными, и, как и выше было написано, много того, что к продуктам не относится совсем.

Наблюдаем, что биграммы не дают много информации о том, как пользователи относятся к тому или иному товару. Этому не благоприятствует наличие у каждой ИГ артикля , т.е. надо было их игнорировать, а также сложные составные названия. 

Возможно, триграммы подойдут под формат отчета лучше:

In [30]:
finder3.nbest(trigram_measures.student_t, 20)

[('is', 'an', 'excellent'),
 ('this', 'sports', 'bra'),
 ('a', 'second', 'pair'),
 ('a', 'quality', 'product'),
 ('a', 'sports', 'bra'),
 ('a', 'second', 'one'),
 ('my', 'second', 'pair'),
 ('the', 'right', 'size'),
 ('best', 'sports', 'bra'),
 ('is', 'an', 'awesome'),
 ('this', 'back', 'pack'),
 ('a', 'size', 'medium'),
 ('is', 'light', 'weight'),
 ('a', 'cup', 'size'),
 ('these', 'boxer', 'briefs'),
 ('the', 'second', 'pair'),
 ('my', 'favorite', 'bra'),
 ('received', 'an', 'email'),
 ('my', 'third', 'pair'),
 ('great', 'sports', 'bra')]

Как и в случае с биграммами, высоко ранжируются коллокации с артиклями, которые не дают нам никакой информации. Однако уже есть прилагательные, но, из оценочного - только good. Другие относятся скорее к характеристикам самого товара, или являются порядковыми числительными.

In [31]:
finder3.nbest(trigram_measures.pmi, 20)

[('layered', 'barrier', 'clones'),
 ('recycled', 'drink', 'boxes'),
 ('YOUR', 'PRODUCTS', 'EVER'),
 ('cheaper', 'knock', 'offs'),
 ('wally', 'world', 'specials'),
 ('disposable', 'foil', 'pan'),
 ('country', 'music', 'festival'),
 ('happy', 'tree', 'friends'),
 ('vintage', 'paratrooper', 'fatigues'),
 ('wooden', 'train', 'whistle'),
 ('block', 'check', 'pattern'),
 ('rough', 'kid', 'son'),
 ('upper', 'arm', 'decoration'),
 ('brief', 'history', 'lesson'),
 ('receiving', 'food', 'aid'),
 ('square', 'cardboard', 'box'),
 ('darker', 'wash', 'denimn'),
 ('rabbit', 'fur', 'poncho'),
 ('stainless', 'steel', 'cable'),
 ('fancy', 'store', 'boughtshirt')]

In [32]:
finder3.nbest(trigram_measures.chi_sq, 20)

[('layered', 'barrier', 'clones'),
 ('recycled', 'drink', 'boxes'),
 ('YOUR', 'PRODUCTS', 'EVER'),
 ('cheaper', 'knock', 'offs'),
 ('wally', 'world', 'specials'),
 ('disposable', 'foil', 'pan'),
 ('happy', 'tree', 'friends'),
 ('wooden', 'train', 'whistle'),
 ('country', 'music', 'festival'),
 ('vintage', 'paratrooper', 'fatigues'),
 ('block', 'check', 'pattern'),
 ('upper', 'arm', 'decoration'),
 ('rough', 'kid', 'son'),
 ('brief', 'history', 'lesson'),
 ('receiving', 'food', 'aid'),
 ('square', 'cardboard', 'box'),
 ('darker', 'wash', 'denimn'),
 ('rabbit', 'fur', 'poncho'),
 ('stainless', 'steel', 'cable'),
 ('fancy', 'store', 'boughtshirt')]

Метрики PMI и хи-квадрат выдают похожие результаты, в топе одинаковые коллокации. Визуально они выглядят подходящими под отчет, потому что присутствуют не только банальные оценочные прилагательные. Но снова попадает достаточно много бессмыслицы, которая никак не относится к товарам одежды и аксессуаров. 

В целом, кажется, что t-test для триграмм показывает самый хороший и интерпретируемый результат, несмотря на то, что топ20 не является очень показательным (ниже топ-50 уже выглядит отлично, и действительно хорошо демонстрирует, понравился ли пользователю товар). Лишние сущности достаточно просто отследить на этом этапе, и убрать их вручную из рассмотрения на одном из предыдущих этапов.

In [33]:
finder3.nbest(trigram_measures.student_t, 50)

[('is', 'an', 'excellent'),
 ('this', 'sports', 'bra'),
 ('a', 'second', 'pair'),
 ('a', 'quality', 'product'),
 ('a', 'sports', 'bra'),
 ('a', 'second', 'one'),
 ('my', 'second', 'pair'),
 ('the', 'right', 'size'),
 ('best', 'sports', 'bra'),
 ('is', 'an', 'awesome'),
 ('this', 'back', 'pack'),
 ('a', 'size', 'medium'),
 ('is', 'light', 'weight'),
 ('a', 'cup', 'size'),
 ('these', 'boxer', 'briefs'),
 ('the', 'second', 'pair'),
 ('my', 'favorite', 'bra'),
 ('received', 'an', 'email'),
 ('my', 'third', 'pair'),
 ('great', 'sports', 'bra'),
 ('the', 'second', 'time'),
 ('is', 'an', 'amazing'),
 ('good', 'quality', 'product'),
 ('this', 'money', 'belt'),
 ('a', 'tank', 'top'),
 ('front', 'closure', 'bras'),
 ('is', 'an', 'adorable'),
 ('first', 'sports', 'bra'),
 ('this', 'style', 'bra'),
 ('the', 'second', 'one'),
 ('credit', 'card', 'holder'),
 ('a', 'snug', 'fit'),
 ('a', 'couple', 'pairs'),
 ('a', 'cute', 'costume'),
 ('good', 'quality', 'shirt'),
 ('a', 'strapless', 'bra'),
 ('the',

In [37]:
ranked_collocations = [' '.join(x[0]) for x in finder3.score_ngrams(trigram_measures.student_t)]

##5

In [70]:
grouped_cols = {}

for entity in products:
  grouped_cols[entity] = []
  for col in ranked_collocations:
    if len(entity.split())>0:
      if entity in col:
        if col not in grouped_cols[entity]:
          grouped_cols[entity].append(col)
    else:
      if entity in col.split():
        if col not in grouped_cols[entity]:
          grouped_cols[entity].append(col)

In [81]:
random5 = np.random.randint(len(products), size=5)

for i in random5:
  print(products[i])
  print('- - - - -')
  for collocation in grouped_cols[products[i]]:
    print(collocation)
  print('- - - - -')

bra
- - - - -
this sports bra
a sports bra
best sports bra
my favorite bra
great sports bra
front closure bras
first sports bra
this style bra
a strapless bra
only sports bra
these sports bras
wear sports bras
favorite sports bra
best strapless bra
this nursing bra
normal bra size
perfect sports bra
a nursing bra
these bra extenders
wear underwire bras
best nursing bra
this model bra
other sports bra
best fitting bra
full coverage bras
other nursing bras
first strapless bra
wear strapless bras
wrong bra size
another sports bra
this sleep bra
my sports bras
a sleep bra
my bra size
the sports bra
a bra size
full coverage bra
brand name hat
different brand bra
best sport bra
only strapless bra
the name brand
usual bra size
best maternity bra
best support bra
model sports bra
regular bra size
an underwire bra
excellent sports bra
good sports bra
my second bra
MANY nursing bras
front hook bras
those bra extenders
Love cotton bras
new sport bras
liked cotton bras
several nursing bras
high im